In [1]:
from pprint import pprint
from pandas_datareader.famafrench import get_available_datasets
import pandas as pd
from statsmodels.api import OLS, add_constant
import pandas_datareader.data as web
from pathlib import Path
from linearmodels.asset_pricing import TradedFactorModel, LinearFactorModel, LinearFactorModelGMM

## Get Data

### Fama-French Risk Premia

#### pandas data_reader

In [2]:
ff_factor = 'F-F_Research_Data_5_Factors_2x3_daily'
ff_data = web.DataReader(ff_factor, 'famafrench')[0]

In [3]:
data_path = Path('..', '00_data')

#### From HDF

In [11]:
with pd.HDFStore(data_path / 'risk_factors.h5') as store:
    ff_data = store.get(ff_factor).tz_localize('UTC')

In [13]:
ff_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2117 entries, 2010-01-04 to 2018-05-31
Data columns (total 6 columns):
Mkt-RF    2117 non-null float64
SMB       2117 non-null float64
HML       2117 non-null float64
RMW       2117 non-null float64
CMA       2117 non-null float64
RF        2117 non-null float64
dtypes: float64(6)
memory usage: 115.8 KB


### Stock Prices

In [14]:
with pd.HDFStore(data_path / 'assets.h5') as store:
    prices = store['quandl/wiki/prices'].sort_index()
    top_vol = prices.loc['2018', 'adj_volume'].unstack().median().nlargest(10)
    returns = prices.adj_close.unstack().filter(top_vol.index).loc['2017':].pct_change().dropna(how='all')
    returns = returns.tz_localize('UTC').dropna(axis=1, thresh=int(len(returns) * .9)).dropna(thresh=int(returns.shape[1] * .9))
    returns = returns.drop(ff_data.columns.intersection(returns.columns), axis=1)
    returns = returns.fillna(returns.median())

### Align

In [15]:
common_dates = ff_data.index.intersection(returns.index)
returns = returns.loc[common_dates]
ff_data = ff_data.loc[common_dates]

In [16]:
ff_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 307 entries, 2017-01-04 to 2018-03-27
Data columns (total 6 columns):
Mkt-RF    307 non-null float64
SMB       307 non-null float64
HML       307 non-null float64
RMW       307 non-null float64
CMA       307 non-null float64
RF        307 non-null float64
dtypes: float64(6)
memory usage: 16.8 KB


In [22]:
ff_data.mean()

Mkt-RF    0.056678
SMB      -0.010098
HML      -0.043420
RMW       0.017264
CMA      -0.040684
RF        0.003704
dtype: float64

### Stocks to zero-mean

In [17]:
returns = returns.sub(ff_data.RF, axis=0)
returns.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 307 entries, 2017-01-04 to 2018-03-27
Data columns (total 10 columns):
GE      307 non-null float64
BAC     307 non-null float64
AMD     307 non-null float64
MU      307 non-null float64
F       307 non-null float64
AAPL    307 non-null float64
CHK     307 non-null float64
INTC    307 non-null float64
MSFT    307 non-null float64
T       307 non-null float64
dtypes: float64(10)
memory usage: 26.4 KB


## Factor Models

### TradedFactorModel

In [18]:
trained_model = TradedFactorModel(returns, ff_data).fit()

In [19]:
trained_model

TradedFactorModel Estimation Summary                      
================================================================================
No. Test Portfolios:                 10   R-squared:                      0.3132
No. Factors:                          6   J-statistic:                    11.816
No. Observations:                   307   P-value                         0.2976
Date:                  Tue, Aug 14 2018   Distribution:                 chi2(10)
Time:                          18:43:34                                         
Cov. Estimator:                  robust                                         
                                                                                
                            Risk Premia Estimates                             
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Mkt-RF         0.0567     0.0389     1.4576     0.1449     -0.0195      0.1329
SMB           -0.0101     0.0275    -0.3668     0.7138     -0.0641      0.0439
HML           -0.0434     0.0265    -1.6377     0.1015     -0.0954      0.0085
RMW            0.0173     0.0171     1.0111     0.3120     -0.0162      0.0507
CMA           -0.0407     0.0181    -2.2455     0.0247     -0.0762     -0.0052
RF             0.0037  7.912e-05     46.812     0.0000      0.0035      0.0039
==============================================================================

Covariance estimator:
HeteroskedasticCovariance
See full_summary for complete results
LinearFactorModelResults, id: 0x7f690831f208

In [20]:
trained_model.full_summary

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                      TradedFactorModel Estimation Summary                      
================================================================================
No. Test Portfolios:                 10   R-squared:                      0.3132
No. Factors:                          6   J-statistic:                    11.816
No. Observations:                   307   P-value                         0.2976
Date:                  Tue, Aug 14 2018   Distribution:                 chi2(10)
Time:                          18:43:34                                         
Cov. Estimator:                  robust                                         
                                                                                
                            Risk Premia Estimates                             
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Mkt-RF         0.0567     0.0389     1.4576     0.1449     -0.0195      0.1329
SMB           -0.0101     0.0275    -0.3668     0.7138     -0.0641      0.0439
HML           -0.0434     0.0265    -1.6377     0.1015     -0.0954      0.0085
RMW            0.0173     0.0171     1.0111     0.3120     -0.0162      0.0507
CMA           -0.0407     0.0181    -2.2455     0.0247     -0.0762     -0.0052
RF             0.0037  7.912e-05     46.812     0.0000      0.0035      0.0039


                               GE Coefficients                                
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
alpha          0.0003     0.0017     0.1466     0.8834     -0.0032      0.0037
Mkt-RF         0.0084     0.0013     6.3381     0.0000      0.0058      0.0109
SMB           -0.0019     0.0014    -1.3818     1.8330     -0.0047      0.0008
HML            0.0009     0.0018     0.4862     0.6269     -0.0027      0.0045
RMW           -0.0031     0.0029    -1.1041     1.7305     -0.0087      0.0024
CMA            0.0131     0.0034     3.8670     0.0001      0.0065      0.0197
RF            -1.7281     0.5453    -3.1692     1.9985     -2.7968     -0.6594


                               BAC Coefficients                               
==============================================================================
alpha          0.0013     0.0012     1.0852     0.2778     -0.0010      0.0036
Mkt-RF         0.0134     0.0007     18.694     0.0000      0.0120      0.0148
SMB            0.0011     0.0009     1.2467     0.2125     -0.0006      0.0028
HML            0.0180     0.0011     15.907     0.0000      0.0158      0.0202
RMW           -0.0016     0.0017    -0.9842     1.6750     -0.0049      0.0016
CMA           -0.0119     0.0017    -6.8827     2.0000     -0.0153     -0.0085
RF            -1.1817     0.2865    -4.1245     2.0000     -1.7433     -0.6202


                               AMD Coefficients                               
==============================================================================
alpha          0.0024     0.0056     0.4273     0.6691     -0.0086      0.0134
Mkt-RF         0.0142     0.0026     5.4554     0.0000      0.0091      0.0194
SMB            0.0063     0.0037     1.6991     0.0893     -0.0010      0.0136
HML           -0.0151     0.0042    -3.5905     1.9997     -0.0234     -0.0069
RMW           -0.0199     0.0070    -2.8395     1.9955     -0.0336     -0.0062
CMA           -0.0035     0.0073    -0.4784     1.3676     -0.0179      0.0109
RF            -1.9316     1.2687    -1.5224     1.8721     -4.4183      0.5551


                               MU Coefficients                                
=================================================

In [21]:
trained_model.betas

,Mkt-RF,SMB,HML,RMW,CMA,RF
GE,0.008355,-0.001936,0.000897,-0.003147,0.013086,-1.728073
BAC,0.013406,0.001107,0.018028,-0.001644,-0.011920,-1.181719
AMD,0.014239,0.006336,-0.015106,-0.019866,-0.003515,-1.931600
MU,0.015764,0.005132,-0.008973,-0.009327,-0.016077,-0.132545
F,0.009847,0.002969,0.004534,0.001664,0.004311,-0.761530
AAPL,0.010863,-0.002903,-0.003145,-0.003788,-0.007723,-1.297088
CHK,0.011788,0.001385,-0.002212,-0.027146,0.038987,-1.380933
INTC,0.012932,-0.001303,-0.001558,0.004113,-0.006067,0.243472
MSFT,0.013119,-0.002608,-0.003905,-0.001890,-0.007520,-0.625373
T,0.007442,-0.001696,0.002020,0.004338,0.005860,-0.916556


### LinearFactorModel

In [24]:
trained_model = LinearFactorModel(returns, ff_data).fit()
trained_model

LinearFactorModel Estimation Summary                      
================================================================================
No. Test Portfolios:                 10   R-squared:                      0.3132
No. Factors:                          6   J-statistic:                    1.9602
No. Observations:                   307   P-value                         0.7431
Date:                  Tue, Aug 14 2018   Distribution:                  chi2(4)
Time:                          18:46:56                                         
Cov. Estimator:                  robust                                         
                                                                                
                            Risk Premia Estimates                             
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Mkt-RF        -0.2142     0.0889    -2.4079     0.0160     -0.3885     -0.0398
SMB           -0.0673     0.2182    -0.3082     0.7579     -0.4949      0.3604
HML            0.0261     0.0740     0.3523     0.7246     -0.1190      0.1711
RMW           -0.1537     0.0810    -1.8978     0.0577     -0.3125      0.0050
CMA           -0.1321     0.0560    -2.3604     0.0183     -0.2418     -0.0224
RF             0.0016     0.0011     1.5408     0.1234     -0.0004      0.0037
==============================================================================

Covariance estimator:
HeteroskedasticCovariance
See full_summary for complete results
LinearFactorModelResults, id: 0x7f6908805f28

### LinearFactorModelGMM

In [23]:
trained_model = LinearFactorModelGMM(returns, ff_data).fit()
trained_model

Iteration: 0, Objective: 3.3288247891491083
Iteration: 10, Objective: 2.065351888881447
Iteration: 20, Objective: 1.9379001954729635
Iteration: 30, Objective: 1.8262047233438226
Iteration: 40, Objective: 1.7899600294728861
Iteration: 50, Objective: 1.714287509739151
Iteration: 60, Objective: 1.532684385549661
Iteration: 70, Objective: 0.7013252151953129
         Current function value: 0.687856
         Iterations: 77
         Function evaluations: 10298
         Gradient evaluations: 139
Iteration: 0, Objective: 0.6911215374918552
Iteration: 10, Objective: 0.690234201265065
Iteration: 20, Objective: 0.6901160949060648
Iteration: 30, Objective: 0.6900522951986282
Iteration: 40, Objective: 0.6899873988154755
Iteration: 50, Objective: 0.6899045704335532
Iteration: 60, Objective: 0.6897852832669381
Iteration: 70, Objective: 0.689391592223882
         Current function value: 0.689386
         Iterations: 76
         Function evaluations: 11408
         Gradient evaluations: 154


LinearFactorModelGMM Estimation Summary                     
================================================================================
No. Test Portfolios:                 10   R-squared:                      0.3130
No. Factors:                          6   J-statistic:                    0.6894
No. Observations:                   307   P-value                         0.9526
Date:                  Tue, Aug 14 2018   Distribution:                  chi2(4)
Time:                          18:45:57                                         
Cov. Estimator:                  robust                                         
                                                                                
                            Risk Premia Estimates                             
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Mkt-RF        -0.2147     0.0729    -2.9440     0.0032     -0.3576     -0.0718
SMB           -0.0511     0.2006    -0.2548     0.7989     -0.4442      0.3420
HML            0.0126     0.0651     0.1929     0.8470     -0.1151      0.1402
RMW           -0.1487     0.0788    -1.8863     0.0593     -0.3032      0.0058
CMA           -0.1452     0.0511    -2.8435     0.0045     -0.2453     -0.0451
RF             0.0016     0.0008     2.0031     0.0452   3.454e-05      0.0032
==============================================================================

Covariance estimator:
HeteroskedasticCovariance
See full_summary for complete results
GMMFactorModelResults, id: 0x7f69088054e0